In [1]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import numpy as np
import cv2
import os

In [2]:
prototxtPath=os.path.sep.join([r'./','deploy.prototxt'])
weightsPath=os.path.sep.join([r'./','res10_300x300_ssd_iter_140000.caffemodel'])


In [3]:
prototxtPath

'./\\deploy.prototxt'

In [4]:
weightsPath

'./\\res10_300x300_ssd_iter_140000.caffemodel'

In [5]:
net=cv2.dnn.readNet(prototxtPath,weightsPath)

In [6]:
model=load_model(r'./mobilenet_v2.model')

In [7]:
image=cv2.imread(r'./Example Images/example_04.jpg')

In [8]:
image

array([[[229, 218, 196],
        [229, 218, 196],
        [229, 218, 196],
        ...,
        [208, 188, 163],
        [208, 188, 163],
        [208, 188, 163]],

       [[230, 219, 197],
        [230, 219, 197],
        [230, 219, 197],
        ...,
        [209, 189, 164],
        [209, 189, 164],
        [209, 189, 164]],

       [[231, 220, 198],
        [231, 220, 198],
        [231, 220, 198],
        ...,
        [211, 191, 166],
        [211, 191, 166],
        [211, 191, 166]],

       ...,

       [[108,  64,  23],
        [117,  74,  31],
        [132,  87,  44],
        ...,
        [167, 146, 125],
        [167, 146, 125],
        [167, 146, 125]],

       [[ 99,  55,  14],
        [109,  66,  23],
        [124,  79,  36],
        ...,
        [165, 144, 123],
        [165, 144, 123],
        [165, 144, 123]],

       [[ 97,  53,  12],
        [108,  65,  22],
        [126,  78,  36],
        ...,
        [164, 143, 122],
        [164, 143, 122],
        [164, 143, 122]]

In [9]:
(h,w)=image.shape[:2]

In [10]:
(h,w)

(408, 612)

In [11]:
blob=cv2.dnn.blobFromImage(image,1.0,(300,300),(104.0,177.0,123.0))

In [12]:
blob

array([[[[ 125.,  125.,  125., ...,  106.,  104.,  104.],
         [ 126.,  126.,  126., ...,  108.,  106.,  106.],
         [ 129.,  129.,  129., ...,  110.,  109.,  109.],
         ...,
         [  10.,   34.,   55., ...,   66.,   65.,   65.],
         [   5.,   30.,   48., ...,   63.,   62.,   62.],
         [  -1.,   27.,   45., ...,   59.,   60.,   60.]],

        [[  41.,   41.,   41., ...,   13.,   11.,   11.],
         [  42.,   42.,   42., ...,   15.,   13.,   13.],
         [  45.,   45.,   45., ...,   17.,   16.,   16.],
         ...,
         [-104.,  -82.,  -64., ...,  -29.,  -29.,  -29.],
         [-112.,  -89.,  -73., ...,  -32.,  -32.,  -32.],
         [-118.,  -93.,  -77., ...,  -36.,  -34.,  -34.]],

        [[  73.,   73.,   73., ...,   42.,   40.,   40.],
         [  74.,   74.,   74., ...,   44.,   42.,   42.],
         [  77.,   77.,   77., ...,   46.,   45.,   45.],
         ...,
         [ -89.,  -71.,  -56., ...,    7.,    4.,    4.],
         [-100.,  -80.,  -

In [13]:
blob.shape

(1, 3, 300, 300)

In [14]:
net.setInput(blob)
detections=net.forward()

In [15]:
detections

array([[[[0.        , 1.        , 0.9975535 , ..., 0.11296627,
          0.5337728 , 0.76351976],
         [0.        , 1.        , 0.14875261, ..., 0.39725092,
          0.7498472 , 1.1502688 ],
         [0.        , 1.        , 0.12599725, ..., 4.0076222 ,
          4.8433027 , 4.982023  ],
         ...,
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ]]]], dtype=float32)

In [16]:
#loop over the detections
for i in range(0,detections.shape[2]):
    confidence=detections[0,0,i,2]
    
    
    if confidence>0.5:
        #we need the X,Y coordinates
        box=detections[0,0,i,3:7]*np.array([w,h,w,h])
        (startX,startY,endX,endY)=box.astype('int')
        
        #ensure the bounding boxes fall within the dimensions of the frame
        (startX,startY)=(max(0,startX),max(0,startY))
        (endX,endY)=(min(w-1,endX), min(h-1,endY))
        
        
        #extract the face ROI, convert it from BGR to RGB channel, resize it to 224,224 and preprocess it
        face=image[startY:endY, startX:endX]
        face=cv2.cvtColor(face,cv2.COLOR_BGR2RGB)
        face=cv2.resize(face,(224,224))
        face=img_to_array(face)
        face=preprocess_input(face)
        face=np.expand_dims(face,axis=0)
        
        (mask,withoutMask,wearedIncorrect)=model.predict(face)[0]
        
        print("Mask:",mask)
        print("No Mask:",withoutMask)
        print("Weared Incorrectly:",wearedIncorrect)

        #determine the class label and color we will use to draw the bounding box and text
        if mask>withoutMask and mask>wearedIncorrect:
            label='Mask'
            color=(0,255,0)
        elif withoutMask>mask and withoutMask>wearedIncorrect:
            label='No mask'
            color=(0,0,255)
        else:
            label='Mask Weared Incorrectly'
            color=(255,0,0)
        
        #include the probability in the label
        label="{}: {:.2f}%".format(label,max(mask,withoutMask,wearedIncorrect)*100)
        
        #display the label and bounding boxes
        cv2.putText(image,label,(startX,startY-10),cv2.FONT_HERSHEY_SIMPLEX,0.45,color,2)
        cv2.rectangle(image,(startX,startY),(endX,endY),color,2)
        
cv2.imshow("OutPut",image)
cv2.waitKey(0)
cv2.destroyAllWindows()

Mask: 0.20030244
No Mask: 0.76538193
Weared Incorrectly: 0.03431567
